<a href="https://colab.research.google.com/github/salanne/dynamol/blob/main/gpaw_convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Running DFT with ASE and GPAW ##

Installing Atomic Simulation Environment (ASE) and GPAW

In [ ]:
%%capture
!apt install python3-mpi4py cython3 libxc-dev gpaw-data
!pip -q install gpaw

Calculating energy of H$_2$

In [ ]:
from ase import Atoms
from ase.io import read, write
from gpaw import GPAW, PW
import time

h2 = Atoms('H2', [(0, 0, 0), (0, 0, 0.74)])
h2.center(vacuum=2.5)
write('h2.cif', h2)
print(h2.cell)
print(h2.positions)

Testing for energy cutoff convergence

In [ ]:
cutoffs = [200,300,400,500,1000]
times = []
energies = []

for cutoff in cutoffs:
  start_time = time.time()
  calc = GPAW(xc='LDA',
              kpts=(1,1,1),
              mode=PW(cutoff),
              txt='h2.txt')
  h2.set_calculator(calc)
  energies.append(h2.get_potential_energy())
  times.append(time.time() - start_time)
print(times)  # seconds
print(energies)  # eV

In [ ]:
import matplotlib.pyplot as plt

f, ax = plt.subplots(2, sharex=True)
ax[0].plot(cutoffs, energies)
ax[0].set_ylabel('Energy (eV)')
ax[0].set_title('Convergence test for H2')

ax[1].plot(cutoffs, times)
ax[1].set_ylabel('Time (s)')
ax[1].set_xlabel('Energy Cutoff (eV)')

plt.show()

Calculating the electron density and plotting a contour plot

In [ ]:
import numpy as np

n = calc.get_all_electron_density(gridrefinement=4)

nred = np.sum(n, axis=0) #sum over density in x-direction
#print(np.shape(nred))

fig, ax = plt.subplots()
ax.contourf(nred)
ax.set_xlim([45, 120])
ax.set_ylim([35, 110])
ax.set_aspect(1)
plt.show()